# Last step
# Predict segmentations

In [1]:
%matplotlib inline

In [2]:
import os
import os.path

import glob

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.morphology

import tqdm

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder

import warnings

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Configurable parameter:
* config (experiment)
* GPU
* input folder 
* output folder

# Configuration

In [3]:
GPU_NO = "2"
input_directory =  "/storage/data/2018_tim_tracking/2018_06_29_lineage_tracking_blainey/images/movies/12_2018_MCF10_Drug_full/images"
output_directory = "/storage/data/2018_tim_tracking/2018_06_29_lineage_tracking_blainey/images/movies/201902/predictions_08"
experiment_name = "08"

# Initialize keras 

In [4]:
from config import config_vars

# load configuration file
config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

# output directory
config_vars["probmap_out_dir"] = output_directory

# initialize GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = GPU_NO

session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)



# Load and preprocessing images

In [5]:
def load_and_prepare_images_from_directory(input_directory):
    image_names = glob.glob(input_directory + "/*png")
    images = []
    imagebuffer = skimage.io.imread_collection( image_names )
    for image in imagebuffer:
        #image = skimage.io.imread(filename)
        images.append(skimage.color.rgb2gray(image))

    #print("found {} images".format(len(imagebuffer)))

    images = np.array(images)

    dim1 = np.floor(images.shape[1]/16) * 16 
    dim1 = dim1.astype(np.int)
    dim2 = np.floor(images.shape[2]/16) * 16 
    dim2 = dim2.astype(np.int)

    images = images[:,0:dim1,0:dim2]

    dim1 = images.shape[1]
    dim2 = images.shape[2]

    images = images.reshape((-1, dim1, dim2, 1))

    #print(dim1,dim2)

    # preprocess images
    percentile = 99.9
    for image_no in range(images.shape[0]):
        orig_img = images[image_no,:,:,:]

        high = np.percentile(orig_img, percentile)
        low = np.percentile(orig_img, 100-percentile)

        img = np.minimum(high, orig_img)
        img = np.maximum(low, img)
        img = (img - low) / (high - low) 
        img = skimage.img_as_ubyte(img) 
        images[image_no,:,:,:] = img # gives float64, thus cast to 8 bit later



    images = images.astype(float)
    images = images / 256

    return(images,imagebuffer)

# Predict images 

for directory in tqdm.tqdm(os.listdir(input_directory)):
    
    if os.path.exists(os.path.join(output_directory,directory)):
        print("Folder {} processed!".format(directory))
    else:
        
    
        [images,imagebuffer] = load_and_prepare_images_from_directory(os.path.join(input_directory,directory))

        # build model and load weights
        dim1 = images.shape[1]
        dim2 = images.shape[2]
        
        
        model = utils.model_builder.get_model_3_class(dim1, dim2)
        model.load_weights(config_vars["model_file"])

        #  prediction 
        predictions = model.predict(images, batch_size=1)


        os.makedirs(os.path.join(output_directory,directory)) 
        for i in range(len(images)):

            image_savename = os.path.join(
                output_directory, 
                directory, 
                os.path.basename(imagebuffer.files[i])
            )

            probmap = predictions[i].squeeze()

            skimage.io.imsave(os.path.splitext(image_savename)[0] + ".png", probmap)

    
    

In [ ]:
print("building model and loading weights")
dim1 = 1024
dim2 = 2048
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.load_weights(config_vars["model_file"])

for directory in os.listdir(input_directory):
    
    #if os.path.exists(os.path.join(output_directory,directory)):
    #    print("Folder {} processed!".format(directory))
    #else:
        
        print("loading images in folder  {}".format(directory))
        [images,imagebuffer] = load_and_prepare_images_from_directory(os.path.join(input_directory,directory))

        if os.path.exists(os.path.join(output_directory,directory)):
            print("Folder exists, overwriting predictions".format(directory))
        else:
            os.makedirs(os.path.join(output_directory,directory))
            
        for i in tqdm.tqdm(range(images.shape[0])):
            image = images[i,0:1024,:,:]
            image = image.reshape(-1, image.shape[0],image.shape[1],1)
            prediction = model.predict(image, batch_size=1)
            
            probmap = prediction.squeeze()
                
            image_savename = os.path.join(
                output_directory, 
                directory, 
                os.path.basename(imagebuffer.files[i])
            )
            with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    skimage.io.imsave(os.path.splitext(image_savename)[0] + ".png", probmap)

        

building model and loading weights


/home/cells2numbers/unet4neutrophils/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/cells2numbers/unet4neutrophils/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/cells2numbers/unet4neutrophils/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/cells2numbers/unet4neutrophils/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/cells2numbers/unet4neutrophils/utils/model_bui

loading images in folder  6wells_wash_001.nd2 - 6wells_wash_001.nd2 (series 02)


/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
100%|██████████| 145/145 [02:45<00:00,  1.14s/it]


loading images in folder  6wells_wash_001.nd2 - 6wells_wash_001.nd2 (series 06)


...

/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
100%|██████████| 145/145 [02:42<00:00,  1.12s/it]


loading images in folder  6wells_wash_001.nd2 - 6wells_wash_001.nd2 (series 01)


/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
100%|██████████| 145/145 [02:36<00:00,  1.08s/it]


loading images in folder  6wells_wash_001.nd2 - 6wells_wash_001.nd2 (series 03)


/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
 33%|███▎      | 48/145 [00:51<01:43,  1.06s/it]